In [3]:
from sklearn.datasets import load_diabetes

import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
import random
X,y = load_diabetes(return_X_y=True)
print(X.shape)
print(y.shape)

(442, 10)
(442,)


In [2]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=2)

In [8]:
class MBGDRegressor:
    def __init__(self,batch_size, epochs, learning_rate):
        self.intercept_ = None
        self.coef_ = None
        self.epochs = epochs
        self.learning_rate = learning_rate
        self.batch_size = batch_size
        
    def fit(self, X_train, y_train):
        self.intercept_ = 0
        self.coef_ = np.ones(X_train.shape[1])
        
        for i in range (self.epochs):
           for j in range(int(X_train.shape[0]/self.batch_size)):
               
                idx = random.sample(range(X_train.shape[0]),self.batch_size)

                y_hat = np.dot(X_train[idx],self.coef_) + self.intercept_
                
                intercept_der  = -2 * np.mean(y_train[idx] - y_hat)
                self.intercept_ = self.intercept_ - (self.learning_rate * intercept_der)
                
                coef_der = -2 * np.dot((y_train[idx] - y_hat),X_train[idx])
                self.coef_ = self.coef_ - (self.learning_rate * coef_der)
        
        print('intercept:',self.intercept_,'coeff:',self.coef_)
  
           
    def predict(self, X_test):
        return np.dot(X_test,self.coef_) + self.intercept_


In [9]:
mbr = MBGDRegressor(batch_size=int(X_train.shape[0]/50),learning_rate=0.01,epochs=100)
mbr.fit(X_train,y_train)

intercept: 148.27393998059975 coeff: [  26.40025583 -145.97724542  451.56608113  308.6841595   -14.79441924
  -88.80042154 -186.12482735  113.44614145  412.60181334  118.44158819]


In [10]:
y_pred = mbr.predict(X_test)
r2_score(y_test,y_pred)

0.44963609631891055

In [11]:
from sklearn.linear_model import SGDRegressor
sgd = SGDRegressor(learning_rate='constant',eta0=0.1)
batch_size = 35

for i in range(100):
    
    idx = random.sample(range(X_train.shape[0]),batch_size)
    sgd.partial_fit(X_train[idx],y_train[idx])
sgd.coef_

array([  46.91066264,  -70.45976401,  361.11601006,  247.08018274,
         17.70737066,  -27.14385367, -166.68285128,  127.49411601,
        305.95605451,  115.93624627])

In [12]:
sgd.intercept_

array([157.09859376])

In [13]:
y_pred = sgd.predict(X_test)
r2_score(y_test,y_pred)

0.43152748711133126